# Training a ConvNet PyTorch

In this notebook, you'll learn how to use the powerful PyTorch framework to specify a conv net architecture and train it on the human action recognition dataset. 


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader,sampler,Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
import timeit
from PIL import Image
import os
import numpy as np
import scipy.io
import pandas as pd

## What's this PyTorch business?

* When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly.
* this notebook will walk you through much of what you need to do to train models using pytorch. if you want to learn more or need further clarification on topics that aren't fully explained here, here are 2 good Pytorch tutorials. 1): http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html 2)http://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* It's not necessary to have a GPU for this homework, using a GPU can make your code run faster.


## Load Datasets

In this part, we will load the action recognition dataset for the neural network. In order to load data from our custom dataset, we need to write a custom Dataloader. If you put hw6_data.mat, /valClips,/trainClips,/testClips under the folder of ./data/ , you do not need to change anything in this part.

First, load the labels of the dataset, you should write your path of the hw6_data.mat file.

In [6]:
label_mat=scipy.io.loadmat('./data/hw6_data.mat')
label_train=label_mat['trLb']
print(len(label_train))
label_val=label_mat['valLb']
print(len(label_val))

7770
2230


### Dataset class

torch.utils.data.Dataset is an abstract class representing a dataset. The custom dataset should inherit Dataset and override the following methods:

    __len__ so that len(dataset) returns the size of the dataset.
    __getitem__ to support the indexing such that dataset[i] can be used to get ith sample

Let’s create a dataset class for our action recognition dataset. We will read images in __getitem__. This is memory efficient because all the images are not stored in the memory at once but read as required.

Sample of our dataset will be a dict {'image':image,'img_path':img_path,'Label':Label}. Our datset will take an optional argument transform so that any required processing can be applied on the sample. 

In [7]:

class ActionDataset(Dataset):
    """Action dataset."""

    def __init__(self,  root_dir,labels=[], transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            labels(list): labels if images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels
    def __len__(self):
        return self.length*3

    def __getitem__(self, idx):
        
        folder=int(idx/3)+1
        imidx=idx%3+1
        folder=format(folder,'05d')
        imgname=str(imidx)+'.jpg'
        img_path = os.path.join(self.root_dir,
                                folder,imgname)
        image = Image.open(img_path)
        if len(self.labels)!=0:
            Label=self.labels[int(idx/3)][0]-1
        if self.transform:
            image = self.transform(image)
        if len(self.labels)!=0:
            sample={'image':image,'img_path':img_path,'Label':Label}
        else:
            sample={'image':image,'img_path':img_path}
        return sample
  

Iterating over the dataset by a for loop.

In [8]:
image_dataset=ActionDataset(root_dir='./data/trainClips/',\
                            labels=label_train,transform=T.ToTensor())

#iterating though the dataset
for i in range(10):
    sample=image_dataset[i]
    print(sample['image'].shape)
    print(sample['Label'])
    print(sample['img_path'])
     
   

torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/1.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/2.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00001/3.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00002/1.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00002/2.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00002/3.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00003/1.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00003/2.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00003/3.jpg
torch.Size([3, 64, 64])
0.0
./data/trainClips/00004/1.jpg


We can iterate over the created dataset with a 'for' loop as before. However, we are losing a lot of features by using a simple for loop to iterate over the data. In particular, we are missing out on:

* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

torch.utils.data.DataLoader is an iterator which provides all these features. 

In [9]:
image_dataloader = DataLoader(image_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i,sample in enumerate(image_dataloader):
    sample['image']=sample['image']
    print(i,sample['image'].shape,sample['img_path'],sample['Label'])
    if i>20: 
        break

0 torch.Size([4, 3, 64, 64]) ['./data/trainClips/06756/1.jpg', './data/trainClips/04659/3.jpg', './data/trainClips/02782/2.jpg', './data/trainClips/01082/2.jpg'] tensor([8., 5., 3., 1.], dtype=torch.float64)
1 torch.Size([4, 3, 64, 64]) ['./data/trainClips/00581/3.jpg', './data/trainClips/04135/3.jpg', './data/trainClips/00339/3.jpg', './data/trainClips/05987/2.jpg'] tensor([0., 4., 0., 7.], dtype=torch.float64)
2 torch.Size([4, 3, 64, 64]) ['./data/trainClips/03090/1.jpg', './data/trainClips/03699/2.jpg', './data/trainClips/05856/3.jpg', './data/trainClips/07200/3.jpg'] tensor([3., 4., 7., 9.], dtype=torch.float64)
3 torch.Size([4, 3, 64, 64]) ['./data/trainClips/06008/3.jpg', './data/trainClips/03333/2.jpg', './data/trainClips/00434/3.jpg', './data/trainClips/04887/3.jpg'] tensor([7., 3., 0., 5.], dtype=torch.float64)
4 torch.Size([4, 3, 64, 64]) ['./data/trainClips/02732/1.jpg', './data/trainClips/04240/2.jpg', './data/trainClips/06602/1.jpg', './data/trainClips/03593/3.jpg'] tensor

Dataloaders for the training, validationg and testing set. 

In [10]:
image_dataset_train=ActionDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

image_dataloader_train = DataLoader(image_dataset_train, batch_size=32,
                        shuffle=True, num_workers=4)
image_dataset_val=ActionDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

image_dataloader_val = DataLoader(image_dataset_val, batch_size=32,
                        shuffle=False, num_workers=4)
image_dataset_test=ActionDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

image_dataloader_test = DataLoader(image_dataset_test, batch_size=32,
                        shuffle=False, num_workers=4)

In [11]:
dtype = torch.FloatTensor # the CPU datatype
# Constant to control how frequently we print train loss
print_every = 100
# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

## Example Model

### Some assorted tidbits

Let's start by looking at a simple model. First, note that PyTorch operates on Tensors, which are n-dimensional arrays functionally analogous to numpy's ndarrays, with the additional feature that they can be used for computations on GPUs.

We'll provide you with a Flatten function, which we explain here. Remember that our image data (and more relevantly, our intermediate feature maps) are initially N x C x H x W, where:
* N is the number of datapoints
* C is the number of image channels. 
* H is the height of the intermediate feature map in pixels
* W is the height of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we input  data into fully connected affine layers, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "Flatten" operation to collapse the C x H x W values per representation into a single long vector. The Flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

In [8]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

### The example model itself

The first step to training your own model is defining its architecture.

Here's an example of a convolutional neural network defined in PyTorch -- try to understand what each line is doing, remembering that each layer is composed upon the previous layer. We haven't trained anything yet - that'll come next - for now, we want you to understand how everything gets set up.  nn.Sequential is a container which applies each layer
one after the other.

In this example, you see 2D convolutional layers (Conv2d), ReLU activations, and fully-connected layers (Linear). You also see the Cross-Entropy loss function, and the Adam optimizer being used. 

Make sure you understand why the parameters of the Linear layer are 10092 and 10.


In [9]:
# Here's where we define the architecture of the model... 
simple_model = nn.Sequential(
                nn.Conv2d(3, 32, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                Flatten(), # see above for explanation
                nn.Linear(26912, 10), # affine layer
              )

# Set the type of all data in this model to be FloatTensor 
simple_model.type(dtype)

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(simple_model.parameters(), lr=1e-2) # lr sets the learning rate of the optimizer

PyTorch supports many other layer types, loss functions, and optimizers - you will experiment with these next. Here's the official API documentation for these (if any of the parameters used above were unclear, this resource will also be helpful). 

* Layers: http://pytorch.org/docs/nn.html
* Activations: http://pytorch.org/docs/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/optim.html#algorithms

## Training a specific model

In this section, we're going to specify a model for you to construct. The goal here isn't to get good performance (that'll be next), but instead to get comfortable with understanding the PyTorch documentation and configuring your own model. 

Using the code provided above as guidance, and using the following PyTorch documentation, specify a model with the following architecture:

* 7x7 Convolutional Layer with 8 filters and stride of 1
* ReLU Activation Layer
* 2x2 Max Pooling layer with a stride of 2
* 7x7 Convolutional Layer with 16 filters and stride of 1
* ReLU Activation Layer
* 2x2 Max Pooling layer with a stride of 2
* Flatten the feature map
* ReLU Activation Layer
* Affine layer to map input units to 10 outputs, you need to figure out the input size here.


In [10]:
fixed_model_base = nn.Sequential( 
    #########1st TODO  (10 points)###################
    nn.Conv2d(3, 8, kernel_size=7, stride=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, stride=2),
    nn.Conv2d(8, 16, kernel_size=7, stride=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(2, stride=2),
    Flatten(),
    nn.ReLU(inplace=True),
    nn.Linear(1936, 10)
    
    ####################################
            )
fixed_model = fixed_model_base.type(dtype)


To make sure you're doing the right thing, use the following tool to check the dimensionality of your output (it should be 32 x 10, since our batches have size 32 and the output of the final affine layer should be 10, corresponding to our 10 classes):

In [11]:
## Now we're going to feed a random batch into the model you defined and make sure the output is the right size
x = torch.randn(32, 3, 64, 64).type(dtype)
x_var = Variable(x.type(dtype)) # Construct a PyTorch Variable out of your input data
ans = fixed_model(x_var)        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
print(np.array(ans.size()))
np.array_equal(np.array(ans.size()), np.array([32, 10]))   


[32 10]


True

### Train the model.

Now that you've seen how to define a model and do a single forward pass of some data through it, let's  walk through how you'd actually train one whole epoch over your training data (using the fixed_model_base we provided above).

Make sure you understand how each PyTorch function used below corresponds to what you implemented in your custom neural network implementation.

Note that because we are not resetting the weights anywhere below, if you run the cell multiple times, you are effectively training multiple epochs (so your performance should improve).

First, set up an RMSprop optimizer (using a 1e-4 learning rate) and a cross-entropy loss function:

In [12]:
################ 2nd TODO  (5 points)##################
optimizer = torch.optim.RMSprop(fixed_model_base.parameters(), lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)
loss_fn = nn.CrossEntropyLoss()


In [13]:
# This sets the model in "training" mode. 
# This is relevant for some layers that may have different behavior
# in training mode vs testing mode, such as Dropout and BatchNorm. 
fixed_model.train()

# Load one batch at a time.
for t, sample in enumerate(image_dataloader_train):
    x_var = Variable(sample['image'])
    #print(type(x_var.data))
    #print(x_var.shape)
    y_var = Variable(sample['Label']).long()

    # This is the forward pass: predict the scores for each class, for each x in the batch.
    scores = fixed_model(x_var)
    
    # Use the correct y values and the predicted y values to compute the loss.
    loss = loss_fn(scores, y_var)
    
    if (t + 1) % print_every == 0:
        print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

    # Zero out all of the gradients for the variables which the optimizer will update.
    optimizer.zero_grad()
    
    # This is the backwards pass: compute the gradient of the loss with respect to each 
    # parameter of the model.
    loss.backward()
    
    # Actually update the parameters of the model using the gradients computed by the backwards pass.
    optimizer.step()
   

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


t = 100, loss = 2.1329
t = 200, loss = 1.8056
t = 300, loss = 1.3633
t = 400, loss = 1.3438
t = 500, loss = 1.3951
t = 600, loss = 1.2720
t = 700, loss = 1.1728


Now you've seen how the training process works in PyTorch. To save you writing boilerplate code, we're providing the following helper functions to help you train for multiple epochs and check the accuracy of your model:

In [12]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'])
            y_var = Variable(sample['Label'].long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        x_var = Variable(sample['image'])
        y_var = sample['Label']
        #y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.max(1)#scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    
    



### Check the accuracy of the model.

Let's see the train and check_accuracy code in action -- feel free to use these methods when evaluating the models you develop below.

You should get a training loss of around 1.0-1.2, and a validation accuracy of around 50-60%. As mentioned above, if you re-run the cells, you'll be training more epochs, so your performance will improve past these numbers.

But don't worry about getting these numbers better -- this was just practice before you tackle designing your own model.

In [15]:
torch.random.manual_seed(12345)
fixed_model.cpu()
fixed_model.apply(reset) 
fixed_model.train() 
train(fixed_model, loss_fn, optimizer,image_dataloader_train, num_epochs=1) 
check_accuracy(fixed_model, image_dataloader_train)# check accuracy on the training set


Starting epoch 1 / 1


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 2.2779
t = 200, loss = 2.2022
t = 300, loss = 2.0407
t = 400, loss = 1.8165
t = 500, loss = 1.7125
t = 600, loss = 1.5291
t = 700, loss = 1.5152
Got 12040 / 23310 correct (51.65)


### Don't forget the validation set!

And note that you can use the check_accuracy function to evaluate on the validation set, by passing **image_dataloader_val** as the second argument to check_accuracy. The accuracy on validation set is arround 40-50%.

In [16]:
check_accuracy(fixed_model, image_dataloader_val)#check accuracy on the validation set

Got 3033 / 6690 correct (45.34)


##### Train a better  model for action recognition!

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition **validation** set. You can use the check_accuracy and train functions from above.

### Things you should try:
- **Filter size**: Above we used 7x7; this makes pretty pictures but smaller filters may be more efficient
- **Number of filters**: Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

### Tips for training
For each network architecture that you try, you should tune the learning rate and regularization strength. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these; however they would be good things to try.

- Alternative update steps: For the assignment we implemented SGD+momentum, RMSprop, and Adam; you could try alternatives like AdaGrad or AdaDelta.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

If you do decide to implement something extra, clearly describe it in the "Extra Credit Description" cell below.

### What we expect
At the very least, you should be able to train a ConvNet that gets at least 55% accuracy on the validation set. This is just a lower bound - if you are careful it should be possible to get accuracies much higher than that! Extra credit points will be awarded for particularly high-scoring models or unique approaches.

You should use the space below to experiment and train your network. 



In [45]:
###########3rd TODO (20 points, must submit the results to Kaggle) ##############
# Train your model here, and make sure the output of this cell is the accuracy of your best model on the 
# train, val, and test sets. Here's some code to get you started. The output of this cell should be the training
# and validation accuracy on your best model (measured by validation accuracy).


# Final Model
model = nn.Sequential( 
                nn.Conv2d(3, 300, kernel_size=10, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, stride = 2),
                nn.BatchNorm2d(300),
                nn.Dropout2d(0.2),
    
                nn.Conv2d(300, 200, kernel_size=5, stride=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(3, stride = 1),
                nn.BatchNorm2d(200),
                nn.Dropout2d(0.2),
    
                nn.Conv2d(200, 100, kernel_size=3, stride=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride=1),
                nn.BatchNorm2d(100),
                nn.Dropout2d(0.2),
    
                nn.Conv2d(100, 50, kernel_size=2, stride=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2, stride=1),
                nn.BatchNorm2d(50),
    
                Flatten(),
    
                nn.Linear(200, 100),
                nn.Linear(100, 50),
                nn.Linear(50, 10),
    
                nn.LogSoftmax()
)

# model = nn.Sequential(
#     nn.Conv2d(3, 300, kernel_size=10, stride=3),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(3, stride = 1),
#     nn.BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     nn.Conv2d(300, 200, kernel_size=6, stride=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(3, stride = 1),
#     nn.BatchNorm2d(200, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     nn.Conv2d(200, 100, kernel_size=3, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 1),
#     nn.BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True),
#     #nn.L1Loss(size_average=True, reduce=True),
    
#     Flatten(),
    
#     nn.Linear(1600, 800),
#     nn.Linear(800, 400),
#     nn.Linear(400, 10),
#     nn.LogSoftmax()
    
# )

# model = nn.Sequential( 
#                 nn.Conv2d(3, 150, kernel_size=8, stride=2),
#                 nn.ReLU(inplace=True),
#                 nn.MaxPool2d(3, stride = 2),
#                 nn.BatchNorm2d(150),
#                 nn.Dropout2d(0.1),
#                 nn.Conv2d(150, 100, kernel_size=5, stride=2),
#                 nn.ReLU(inplace=True),
#                 nn.MaxPool2d(2, stride = 1),
#                 nn.BatchNorm2d(100),
#                 nn.Dropout2d(0.2),
#                 nn.Conv2d(100, 50, kernel_size=2, stride=1),
#                 nn.ReLU(inplace=True),
#                 nn.MaxPool2d(2, stride=1),
#                 nn.BatchNorm2d(50),
#                 Flatten(),
#                 nn.Linear(200, 100),
#                 nn.Linear(100, 50),
#                 nn.Linear(50, 10),
#                 nn.LogSoftmax()
# )

# model = nn.Sequential(
#     nn.Conv2d(3, 100, kernel_size=7, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 2),
#     nn.BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),

#     nn.Conv2d(100, 75, kernel_size=6, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 2),
#     nn.BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),

#     nn.Conv2d(75, 50, kernel_size=5, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 1),
#     nn.BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),

#     Flatten(),
#     nn.ReLU(inplace=True),

#     nn.Linear(800, 10),
#     nn.LogSoftmax() 
    
# )

# model = nn.Sequential(
#     nn.Conv2d(3, 300, kernel_size=5, stride=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(3, stride = 1),
#     nn.BatchNorm2d(300, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     nn.Conv2d(300, 225, kernel_size=4, stride=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(3, stride = 1),
#     nn.BatchNorm2d(225, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     nn.Conv2d(225, 150, kernel_size=3, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 1),
#     nn.BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     nn.Conv2d(150, 75, kernel_size=2, stride=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool2d(2, stride = 1),
#     nn.BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True),
#     nn.L1Loss(size_average=True, reduce=True),
    
#     Flatten(),
#     nn.ReLU(inplace=True),
#     nn.Linear(675, 10),
#     nn.LogSoftmax() 
    
# )

fixed_model = model.type(dtype)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False)

In [46]:
x = torch.randn(32, 3, 64, 64).type(dtype)
x_var = Variable(x.type(dtype)) 
ans = fixed_model(x_var)
print(np.array(ans.size()))
np.array_equal(np.array(ans.size()), np.array([32, 10]))

[32 10]


/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


True

In [47]:
fixed_model.cpu()
fixed_model.apply(reset) 
fixed_model.train() 
train(fixed_model, loss_fn, optimizer,image_dataloader_train, num_epochs=10) 
check_accuracy(fixed_model, image_dataloader_train)
check_accuracy(fixed_model, image_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.1259
t = 200, loss = 1.0671
t = 300, loss = 0.7296
t = 400, loss = 0.5969
t = 500, loss = 0.7692
t = 600, loss = 0.3980
t = 700, loss = 0.3669
Starting epoch 2 / 10
t = 100, loss = 0.4436
t = 200, loss = 0.2795
t = 300, loss = 0.1845
t = 400, loss = 0.4646
t = 500, loss = 0.2951
t = 600, loss = 0.2789
t = 700, loss = 0.2328
Starting epoch 3 / 10
t = 100, loss = 0.2586
t = 200, loss = 0.3825
t = 300, loss = 0.1833
t = 400, loss = 0.1842
t = 500, loss = 0.1218
t = 600, loss = 0.0515
t = 700, loss = 0.0192
Starting epoch 4 / 10
t = 100, loss = 0.0306
t = 200, loss = 0.0276
t = 300, loss = 0.0985
t = 400, loss = 0.0653
t = 500, loss = 0.0486
t = 600, loss = 0.1782
t = 700, loss = 0.0924
Starting epoch 5 / 10
t = 100, loss = 0.2500
t = 200, loss = 0.0642
t = 300, loss = 0.0774
t = 400, loss = 0.0298
t = 500, loss = 0.0384
t = 600, loss = 0.1420
t = 700, loss = 0.3274
Starting epoch 6 / 10
t = 100, loss = 0.1956
t = 200, loss = 0.2383
t = 300, loss = 0.0271
t = 400, loss = 

### Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and any visualizations or graphs that you make in the process of training and evaluating your network.

# 4th ToDo

In [72]:
# Tell us here!
# ###########4th TODO (5 points) ##############

# For the final model:
# In the "things you should try" text above it was mentioned some architectures to follow. 
# I followed the following architecture: [conv-relu-pool-Norm-dropout]xN -> [affine]xM -> [softmax]
# Other points mentioned were that filter size smaller can be more efficient. 
# Hence, I started with filter size of 10x10 and then reduced it momentarily with a stride of 2 to start with. 
# Then further it was suggested to use Batch Normalization and hence I used it in my final network architecture. 
# Also, as mentioned that we can use either L2 norm or dropout, I used dropout for my architecture.

# My final network architecture looked like:

# 10x10 conv2d layer with 2 strides and 300 filters
# relu activation layer with inplace True
# MaxPool2d layer with 2 strides and 3x3 window size
# BatchNorm2d layer on the data set for 300 filters
# Dropout layer with threshold of 0.2

# 5x5 conv2d layer with 1 stride and 200 filters
# relu activation layer with inplace True
# MaxPool2d layer with 1 stride and 3x3 window size
# BatchNorm2d layer on the data set for 200 filters
# Dropout layer with threshold of 0.2

# 3x3 conv2d layer with 1 stride and 100 filters
# relu activation layer with inplace True
# MaxPool2d layer with 1 stride and 2x2 window size
# BatchNorm2d layer on the data set for 100 filters
# Dropout layer with threshold of 0.2

# 2x2 conv2d layer with 1 stride and 50 filters
# relu activation layer with inplace True
# MaxPool2d layer with 1 stride1 and 2x2 window size
# BatchNorm2d layer on the data set for 50 filters

# Used flatten function

# Used Linear function which is the affine layer with input as 200 to 100
# Used Linear function which is the affine layer with input as 100 to 50
# Used Linear function which is the affine layer with input as 50 to 10

# Used LogSoftmax layer

### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/c/cse512springhw3) to see how well your network performs on the test set. 
#######5th TODO (submit the result to Kaggle, the highest 3 entries get extra 10 points )###############

In [48]:
def predict_on_test(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for t, sample in enumerate(loader):
        x_var = Variable(sample['image'])
        scores = model(x_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count
    
count=predict_on_test(fixed_model, image_dataloader_test)
print(count)

/anaconda3/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


9810


### GPU! (This part is optional, 0 points)

If you have access to GPU, you can make the code run on GPU, it would be much faster. 

Now, we're going to switch the dtype of the model and our data to the GPU-friendly tensors, and see what happens... everything is the same, except we are casting our model and input tensors as this new dtype instead of the old one.

If this returns false, or otherwise fails in a not-graceful way (i.e., with some error message), you may not have an NVIDIA GPU available on your machine. 

In [39]:
# Verify that CUDA is properly configured and you have a GPU available

torch.cuda.is_available()

False

In [ ]:
import copy
gpu_dtype = torch.cuda.FloatTensor

fixed_model_gpu = copy.deepcopy(fixed_model_base)#.type(gpu_dtype)
fixed_model_gpu.cuda()
x_gpu = torch.randn(4, 3, 64, 64).cuda()#.type(gpu_dtype)
x_var_gpu = Variable(x_gpu)#type(gpu_dtype)) # Construct a PyTorch Variable out of your input data
ans = fixed_model_gpu(x_var_gpu)        # Feed it through the model! 

# Check to make sure what comes out of your model
# is the right dimensionality... this should be True
# if you've done everything correctly
np.array_equal(np.array(ans.size()), np.array([4, 10]))


Run the following cell to evaluate the performance of the forward pass running on the CPU:

In [ ]:
%%timeit 
ans = fixed_model(x_var)

... and now the GPU:

In [ ]:
%%timeit 
torch.cuda.synchronize() # Make sure there are no pending GPU computations
ans = fixed_model_gpu(x_var_gpu)        # Feed it through the model! 
torch.cuda.synchronize() # Make sure there are no pending GPU computations

You should observe that even a simple forward pass like this is significantly faster on the GPU. So for the rest of the assignment (and when you go train your models in assignment 3 and your project!), you should use the GPU datatype for your model and your tensors: as a reminder that is *torch.cuda.FloatTensor* (in our notebook here as *gpu_dtype*)

Let's make the loss function and training variables to GPU friendly format by '.cuda()'

In [ ]:
loss_fn = nn.CrossEntropyLoss().cuda()
optimizer = optim.RMSprop(fixed_model_gpu.parameters(), lr=1e-4)

In [ ]:
def train(model, loss_fn, optimizer, dataloader, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['image'].cuda())
            y_var = Variable(sample['Label'].cuda().long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        x_var = Variable(sample['image'].cuda())
        y_var = sample['Label'].cuda()
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

Run on GPU!

In [ ]:
torch.cuda.random.manual_seed(12345)

fixed_model_gpu.apply(reset) 
fixed_model_gpu.train() 
train(fixed_model_gpu, loss_fn, optimizer,image_dataloader_train, num_epochs=1) 
check_accuracy(fixed_model_gpu, image_dataloader_train)# check accuracy on the training set


### 3D Convolution on video clips (25 points+10 extra points)
3D convolution is for videos, it has one more dimension than 2d convolution. You can find the document for 3D convolution here http://pytorch.org/docs/master/nn.html#torch.nn.Conv3dIn. In our dataset, each clip is a video of 3 frames. Lets classify the each clip rather than each image using 3D convolution.
We offer the data loader, the train_3d and check_accuracy

In [13]:
class ActionClipDataset(Dataset):
    """Action Landmarks dataset."""

    def __init__(self,  root_dir,labels=[], transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        
        self.root_dir = root_dir
        self.transform = transform
        self.length=len(os.listdir(self.root_dir))
        self.labels=labels

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        
        folder=idx+1
        folder=format(folder,'05d')
        clip=[]
        if len(self.labels)!=0:
            Label=self.labels[idx][0]-1
        for i in range(3):
            imidx=i+1
            imgname=str(imidx)+'.jpg'
            img_path = os.path.join(self.root_dir,
                                    folder,imgname)
            image = Image.open(img_path)
            image=np.array(image)
            clip.append(image)
        if self.transform:
            clip=np.asarray(clip)
            clip=np.transpose(clip, (0,3,1,2))
            clip = torch.from_numpy(np.asarray(clip))
        if len(self.labels)!=0:
            sample={'clip':clip,'Label':Label,'folder':folder}
        else:
            sample={'clip':clip,'folder':folder}
        return sample

clip_dataset=ActionClipDataset(root_dir='./data/trainClips/',\
                               labels=label_train,transform=T.ToTensor())#/home/tqvinh/Study/CSE512/cse512-s18/hw2data/trainClips/
for i in range(10):
    sample=clip_dataset[i]
    print(sample['clip'].shape)
    print(sample['Label'])
    print(sample['folder'])

torch.Size([3, 3, 64, 64])
0.0
00001
torch.Size([3, 3, 64, 64])
0.0
00002
torch.Size([3, 3, 64, 64])
0.0
00003
torch.Size([3, 3, 64, 64])
0.0
00004
torch.Size([3, 3, 64, 64])
0.0
00005
torch.Size([3, 3, 64, 64])
0.0
00006
torch.Size([3, 3, 64, 64])
0.0
00007
torch.Size([3, 3, 64, 64])
0.0
00008
torch.Size([3, 3, 64, 64])
0.0
00009
torch.Size([3, 3, 64, 64])
0.0
00010


In [14]:
clip_dataloader = DataLoader(clip_dataset, batch_size=4,
                        shuffle=True, num_workers=4)


for i,sample in enumerate(clip_dataloader):
    print(i,sample['clip'].shape,sample['folder'],sample['Label'])
    if i>20: 
        break

0 torch.Size([4, 3, 3, 64, 64]) ['00385', '04253', '05311', '05483'] tensor([0., 5., 6., 6.], dtype=torch.float64)
1 torch.Size([4, 3, 3, 64, 64]) ['06084', '00055', '05293', '07001'] tensor([7., 0., 6., 8.], dtype=torch.float64)
2 torch.Size([4, 3, 3, 64, 64]) ['02943', '07238', '00284', '02527'] tensor([3., 9., 0., 2.], dtype=torch.float64)
3 torch.Size([4, 3, 3, 64, 64]) ['02395', '06047', '03416', '00902'] tensor([2., 7., 3., 0.], dtype=torch.float64)
4 torch.Size([4, 3, 3, 64, 64]) ['06571', '00304', '00477', '03020'] tensor([8., 0., 0., 3.], dtype=torch.float64)
5 torch.Size([4, 3, 3, 64, 64]) ['02637', '05892', '03469', '04322'] tensor([2., 7., 3., 5.], dtype=torch.float64)
6 torch.Size([4, 3, 3, 64, 64]) ['03032', '05550', '02333', '00344'] tensor([3., 6., 2., 0.], dtype=torch.float64)
7 torch.Size([4, 3, 3, 64, 64]) ['07722', '03298', '02042', '05434'] tensor([9., 3., 2., 6.], dtype=torch.float64)
8 torch.Size([4, 3, 3, 64, 64]) ['05202', '03254', '05736', '05508'] tensor([6.,

In [15]:
clip_dataset_train=ActionClipDataset(root_dir='./data/trainClips/',labels=label_train,transform=T.ToTensor())

clip_dataloader_train = DataLoader(clip_dataset_train, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_val=ActionClipDataset(root_dir='./data/valClips/',labels=label_val,transform=T.ToTensor())

clip_dataloader_val = DataLoader(clip_dataset_val, batch_size=16,
                        shuffle=True, num_workers=4)
clip_dataset_test=ActionClipDataset(root_dir='./data/testClips/',labels=[],transform=T.ToTensor())

clip_dataloader_test = DataLoader(clip_dataset_test, batch_size=16,
                        shuffle=False, num_workers=4)

Write the Flatten for 3d covolution feature maps.

In [16]:
class Flatten3d(nn.Module):
    def forward(self, x):
        ###############6th TODO (5 points)###################
        N, C, D, H, W = x.size()
        return x.view(N, -1)
        

Design a network using 3D convolution on videos for video classification.

In [87]:
fixed_model_3d = nn.Sequential( 
    # You fill this in!
    ###############7th TODO (20 points)#########################

# # Sixth Model
#     nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(16),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(16, 128, kernel_size=2, stride=1, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(128),
#     nn.Dropout3d(0.2),
    
    
#     Flatten3d(),
#     nn.ReLU(inplace=True),
    
#     nn.Linear(20736, 10),
    
    
    ########################################################
    
    # Fifth Model - Final model ; as got the best accuracy from this
    nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=2, stride=2),
    nn.BatchNorm3d(16),
    nn.Dropout3d(0.2),
    
    nn.Conv3d(16, 64, kernel_size=2, stride=1, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool3d(kernel_size=2, stride=2),
    nn.BatchNorm3d(64),
    nn.Dropout3d(0.2),
    
    
    Flatten3d(),
    nn.ReLU(inplace=True),
    
    nn.Linear(10368, 10),
    
    ########################################################
    
    
    
    
#     # Fourth Model
#     nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(16),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(16, 32, kernel_size=2, stride=1, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(32),
#     nn.Dropout3d(0.1),
    
    
#     Flatten3d(),
#     nn.ReLU(inplace=True),
    
#     nn.Linear(5184, 432),
#     nn.Linear(432, 36),
#     nn.Linear(36, 10),
    
## Third Model
#     nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(16),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(16, 32, kernel_size=2, stride=1, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(32),
#     nn.Dropout3d(0.2),
    
    
#     Flatten3d(),
#     nn.ReLU(inplace=True),
    
#     nn.Linear(5184, 10),
    
    
## Second Model   
#     nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(16),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(16, 32, kernel_size=2, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=1, stride=1),
#     nn.BatchNorm3d(32),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(32, 64, kernel_size=1, stride=1, padding=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=1, stride=1),
#     nn.BatchNorm3d(64),
#     nn.Dropout3d(0.2),
    
    
#     Flatten3d(),
#     nn.ReLU(inplace=True),
    
#     nn.Linear(36864, 192),
#     nn.Linear(192, 10),
    
    
    
## First Model
#     nn.Conv3d(3, 16, kernel_size=3, stride=2, padding=2),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=2, stride=2),
#     nn.BatchNorm3d(16),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(16, 32, kernel_size=2, stride=2, padding=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=1, stride=2),
#     nn.BatchNorm3d(32),
#     nn.Dropout3d(0.2),
    
#     nn.Conv3d(32, 64, kernel_size=1, stride=1, padding=1),
#     nn.ReLU(inplace=True),
#     nn.MaxPool3d(kernel_size=1, stride=1),
#     nn.BatchNorm3d(64),
#     nn.Dropout3d(0.1),
    
    
#     Flatten3d(),
#     nn.ReLU(inplace=True),
    
#     nn.Linear(9408, 4704),
#     nn.Linear(4704, 2352),
#     nn.Linear(2352, 10),
    
    
)

fixed_model_3d = fixed_model_3d.type(dtype)
x = torch.randn(32,3, 3, 64, 64).type(dtype)
x_var = Variable(x).type(dtype) # Construct a PyTorch Variable out of your input data
ans = fixed_model_3d(x_var) 
np.array_equal(np.array(ans.size()), np.array([32, 10]))


True

### Describe what you did (5 points)

In the cell below you should write an explanation of what you did, any additional features that you implemented, and any visualizations or graphs that you make in the process of training and evaluating your network.

### 8th TODO Tell us here:

In [88]:
# I took reference from my previos model and this time decreased my kernel size and 
# increased the number of filters with each layer

# My model has the following architecture:
# [conv-relu-pool-Norm-dropout]xN -> [affine]xM

# My final network architecture is:

# 3x3 conv3d layer with 2 strides and 16 filters and 2 padding
# relu activation layer with inplace True
# MaxPool3d layer with 2 strides and 2x2 window size
# BatchNorm3d layer on the data set for 16 filters
# Dropout layer with threshold of 0.2

# 2x2 conv3d layer with 1 stride and 64 filters and 2 padding
# relu activation layer with inplace True
# MaxPool3d layer with 2 strides and 2x2 window size
# BatchNorm3d layer on the data set for 64 filters
# Dropout layer with threshold of 0.2

# Used flatten function
# relu activation layer with inplace True

# Used Linear function which is the affine layer with input as 10368 to 10 as output

In [89]:
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.RMSprop(fixed_model_3d.parameters(), lr=1e-4)


In [90]:
def train_3d(model, loss_fn, optimizer,dataloader,num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, sample in enumerate(dataloader):
            x_var = Variable(sample['clip'].type(dtype))
            y_var = Variable(sample['Label'].type(dtype).long())

            scores = model(x_var)
            
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy_3d(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, sample in enumerate(loader):
        x_var = Variable(sample['clip'].type(dtype))
        y_var = sample['Label'].type(dtype)
        y_var=y_var.cpu()
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        #print(preds)
        #print(y_var)
        num_correct += (preds.numpy() == y_var.numpy()).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [91]:
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.2790
t = 200, loss = 0.7527
t = 300, loss = 0.6266
t = 400, loss = 0.5459
Starting epoch 2 / 10
t = 100, loss = 0.2841
t = 200, loss = 0.5181
t = 300, loss = 0.5683
t = 400, loss = 0.3582
Starting epoch 3 / 10
t = 100, loss = 0.3183
t = 200, loss = 0.1477
t = 300, loss = 0.1314
t = 400, loss = 0.3942
Starting epoch 4 / 10
t = 100, loss = 0.1923
t = 200, loss = 0.2524
t = 300, loss = 0.3083
t = 400, loss = 0.2548
Starting epoch 5 / 10
t = 100, loss = 0.0772
t = 200, loss = 0.0353
t = 300, loss = 0.2339
t = 400, loss = 0.1179
Starting epoch 6 / 10
t = 100, loss = 0.1098
t = 200, loss = 0.0302
t = 300, loss = 0.0811
t = 400, loss = 0.0594
Starting epoch 7 / 10
t = 100, loss = 0.0308
t = 200, loss = 0.1362
t = 300, loss = 0.1152
t = 400, loss = 0.0404
Starting epoch 8 / 10
t = 100, loss = 0.0159
t = 200, loss = 0.0309
t = 300, loss = 0.2406
t = 400, loss = 0.1317
Starting epoch 9 / 10
t = 100, loss = 0.0255
t = 200, loss = 0.1025
t = 300, loss = 0.0060
t = 400, loss = 0.0

In [77]:
# Fifth Model - Final model
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.3368
t = 200, loss = 1.0647
t = 300, loss = 1.3216
t = 400, loss = 0.3675
Starting epoch 2 / 10
t = 100, loss = 0.8657
t = 200, loss = 0.5539
t = 300, loss = 0.6878
t = 400, loss = 0.5717
Starting epoch 3 / 10
t = 100, loss = 0.4232
t = 200, loss = 0.1448
t = 300, loss = 0.2247
t = 400, loss = 0.3301
Starting epoch 4 / 10
t = 100, loss = 0.1830
t = 200, loss = 0.1844
t = 300, loss = 0.4027
t = 400, loss = 0.3996
Starting epoch 5 / 10
t = 100, loss = 0.1301
t = 200, loss = 0.2617
t = 300, loss = 0.0720
t = 400, loss = 0.3915
Starting epoch 6 / 10
t = 100, loss = 0.2913
t = 200, loss = 0.1795
t = 300, loss = 0.1360
t = 400, loss = 0.0619
Starting epoch 7 / 10
t = 100, loss = 0.1054
t = 200, loss = 0.0706
t = 300, loss = 0.0799
t = 400, loss = 0.0180
Starting epoch 8 / 10
t = 100, loss = 0.1059
t = 200, loss = 0.1358
t = 300, loss = 0.1191
t = 400, loss = 0.2480
Starting epoch 9 / 10
t = 100, loss = 0.2051
t = 200, loss = 0.1838
t = 300, loss = 0.1516
t = 400, loss = 0.0

In [54]:
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 0.9959
t = 200, loss = 0.9246
t = 300, loss = 0.6494
t = 400, loss = 0.5452
Starting epoch 2 / 10
t = 100, loss = 0.4174
t = 200, loss = 0.3533
t = 300, loss = 0.4003
t = 400, loss = 0.8135
Starting epoch 3 / 10
t = 100, loss = 0.3049
t = 200, loss = 0.7425
t = 300, loss = 0.4531
t = 400, loss = 0.2317
Starting epoch 4 / 10
t = 100, loss = 0.1043
t = 200, loss = 0.4243
t = 300, loss = 0.1719
t = 400, loss = 0.1178
Starting epoch 5 / 10
t = 100, loss = 0.2515
t = 200, loss = 0.3547
t = 300, loss = 0.1463
t = 400, loss = 0.3559
Starting epoch 6 / 10
t = 100, loss = 0.2023
t = 200, loss = 0.0599
t = 300, loss = 0.0532
t = 400, loss = 0.2234
Starting epoch 7 / 10
t = 100, loss = 0.0152
t = 200, loss = 0.1625
t = 300, loss = 0.0327
t = 400, loss = 0.0076
Starting epoch 8 / 10
t = 100, loss = 0.0247
t = 200, loss = 0.0978
t = 300, loss = 0.1313
t = 400, loss = 0.2670
Starting epoch 9 / 10
t = 100, loss = 0.0948
t = 200, loss = 0.0362
t = 300, loss = 0.0283
t = 400, loss = 0.0

In [50]:
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.9300
t = 200, loss = 1.4822
t = 300, loss = 1.0574
t = 400, loss = 1.5905
Starting epoch 2 / 10
t = 100, loss = 1.7069
t = 200, loss = 0.5282
t = 300, loss = 0.5969
t = 400, loss = 1.1318
Starting epoch 3 / 10
t = 100, loss = 1.0177
t = 200, loss = 0.3719
t = 300, loss = 0.6128
t = 400, loss = 0.3986
Starting epoch 4 / 10
t = 100, loss = 0.5080
t = 200, loss = 0.7030
t = 300, loss = 0.7014
t = 400, loss = 0.6665
Starting epoch 5 / 10
t = 100, loss = 0.4112
t = 200, loss = 0.7887
t = 300, loss = 0.4671
t = 400, loss = 0.8185
Starting epoch 6 / 10
t = 100, loss = 1.0331
t = 200, loss = 0.6921
t = 300, loss = 0.3144
t = 400, loss = 0.3010
Starting epoch 7 / 10
t = 100, loss = 0.6325
t = 200, loss = 0.6348
t = 300, loss = 1.4530
t = 400, loss = 0.4719
Starting epoch 8 / 10
t = 100, loss = 0.7725
t = 200, loss = 0.5946
t = 300, loss = 0.5549
t = 400, loss = 0.3875
Starting epoch 9 / 10
t = 100, loss = 0.5215
t = 200, loss = 0.2452
t = 300, loss = 0.7311
t = 400, loss = 0.5

In [23]:
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.4425
t = 200, loss = 1.3213
t = 300, loss = 0.9584
t = 400, loss = 1.4459
Starting epoch 2 / 10
t = 100, loss = 1.0034
t = 200, loss = 0.7845
t = 300, loss = 0.7653
t = 400, loss = 0.9606
Starting epoch 3 / 10
t = 100, loss = 0.6300
t = 200, loss = 0.6967
t = 300, loss = 0.3529
t = 400, loss = 0.2858
Starting epoch 4 / 10
t = 100, loss = 0.7728
t = 200, loss = 0.6766
t = 300, loss = 0.1428
t = 400, loss = 0.4633
Starting epoch 5 / 10
t = 100, loss = 0.2738
t = 200, loss = 0.2696
t = 300, loss = 0.9743
t = 400, loss = 0.3560
Starting epoch 6 / 10
t = 100, loss = 0.1255
t = 200, loss = 0.8760
t = 300, loss = 0.3537
t = 400, loss = 0.2746
Starting epoch 7 / 10
t = 100, loss = 0.2245
t = 200, loss = 0.6381
t = 300, loss = 0.2130
t = 400, loss = 0.1199
Starting epoch 8 / 10
t = 100, loss = 0.4597
t = 200, loss = 0.2227
t = 300, loss = 0.3203
t = 400, loss = 0.2551
Starting epoch 9 / 10
t = 100, loss = 0.0763
t = 200, loss = 0.3842
t = 300, loss = 0.2014
t = 400, loss = 0.2

In [71]:
torch.cuda.random.manual_seed(12345)
fixed_model_3d.apply(reset) 
fixed_model_3d.train() 
train_3d(fixed_model_3d, loss_fn, optimizer,clip_dataloader_train, num_epochs=10) 
fixed_model_3d.eval() 
check_accuracy_3d(fixed_model_3d, clip_dataloader_val)

Starting epoch 1 / 10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]


t = 100, loss = 1.7577
t = 200, loss = 1.5239
t = 300, loss = 1.0293
t = 400, loss = 0.8840
Starting epoch 2 / 10
t = 100, loss = 0.8858
t = 200, loss = 0.7756
t = 300, loss = 0.7024
t = 400, loss = 0.6548
Starting epoch 3 / 10
t = 100, loss = 0.4647
t = 200, loss = 0.4214
t = 300, loss = 0.5107
t = 400, loss = 0.2809
Starting epoch 4 / 10
t = 100, loss = 0.3629
t = 200, loss = 0.2722
t = 300, loss = 0.3906
t = 400, loss = 0.1943
Starting epoch 5 / 10
t = 100, loss = 0.4652
t = 200, loss = 0.3013
t = 300, loss = 0.2516
t = 400, loss = 0.1837
Starting epoch 6 / 10
t = 100, loss = 0.1927
t = 200, loss = 0.1184
t = 300, loss = 0.2952
t = 400, loss = 0.4080
Starting epoch 7 / 10
t = 100, loss = 0.4607
t = 200, loss = 0.3313
t = 300, loss = 0.2440
t = 400, loss = 0.1913
Starting epoch 8 / 10
t = 100, loss = 0.2389
t = 200, loss = 0.1219
t = 300, loss = 0.2175
t = 400, loss = 0.6064
Starting epoch 9 / 10
t = 100, loss = 0.2449
t = 200, loss = 0.0976
t = 300, loss = 0.0978
t = 400, loss = 0.0

Test your 3d convolution model on the validation set. You don't need to submit the result of this part to kaggle.  

Test your model on the test set, predict_on_test_3d() will generate a file named 'results_3d.csv'. Please submit the csv file to kaggle https://www.kaggle.com/c/cse512springhw3video
The highest 3 entries get extra 10 points.


In [94]:
def predict_on_test_3d(model, loader):
    '''
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')  
    '''
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results_3d.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for t, sample in enumerate(loader):
        x_var = Variable(sample['clip'].type(dtype))
        scores = model(x_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count
    
count=predict_on_test_3d(fixed_model_3d, clip_dataloader_test)
print(count)

3270


In [81]:
# Correcting the CSV File for Kaggle Submission

In [95]:
#csv_df = pd.read_csv('results.csv')
csv_df = pd.read_csv('results_3d.csv')
ans = []
for i in range(len(csv_df)):
    ans.append(csv_df['Class'][i][7])
csv_df['Class'] = ans

In [96]:
csv_df.to_csv('/Users/astitvnagpal/Desktop/class_3d.csv', index=None)